# Dependencies

In [45]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [46]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

#Model 1: Bin "Ask Amount" column

##M1: Preprocessing 

In [48]:
# EXPLORE ASK AND TRY BINNING VALUES
amtcounts = application_df["ASK_AMT"].value_counts()
amtcounts

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [49]:
print(application_df["ASK_AMT"].min())
print(application_df["ASK_AMT"].median())
print(application_df["ASK_AMT"].max())

5000
5000.0
8597806340


In [50]:
# of values above 5 million
(application_df["ASK_AMT"] > 5000000).sum()

637

In [51]:
# of values under 5 million & above 1 million
((application_df["ASK_AMT"] > 1000000) & (application_df["ASK_AMT"] < 5000000)).sum()

940

In [52]:
# of values under 1 million & above 100 grand
((application_df["ASK_AMT"] > 100000) & (application_df["ASK_AMT"] < 1000000)).sum()

2954

In [53]:
# of values under 100 grand & above 5000
((application_df["ASK_AMT"] > 5000) & (application_df["ASK_AMT"] < 100000)).sum()

4368

In [54]:
# CREATE BINS 
bins = [0, 5000, 100000, 1000000, 5000000, float('inf')] 
application_df["ASK_AMT"] = pd.cut(application_df["ASK_AMT"], bins=bins)

In [55]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,"(0.0, 5000.0]",1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,"(100000.0, 1000000.0]",1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,"(0.0, 5000.0]",0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,"(5000.0, 100000.0]",1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,"(100000.0, 1000000.0]",1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,"(0.0, 5000.0]",0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,"(0.0, 5000.0]",0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,"(0.0, 5000.0]",0
34297,T5,Independent,C3000,ProductDev,Association,0,"(0.0, 5000.0]",1


In [56]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
ASK_AMT              5
IS_SUCCESSFUL        2
dtype: int64

In [57]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [58]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T17", "T15", "T29", "T14", "T25", "T2", "T12", "T13", "T9"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [59]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [60]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts = application_df['CLASSIFICATION'].value_counts()
counts[counts>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [61]:
counts[counts<1883].index

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278',
       'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600',
       'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256',
       'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150'],
      dtype='object')

In [62]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278',
       'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600',
       'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256',
       'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [63]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df.dtypes


APPLICATION_TYPE      object
AFFILIATION           object
CLASSIFICATION        object
USE_CASE              object
ORGANIZATION          object
INCOME_AMT            object
ASK_AMT             category
IS_SUCCESSFUL          int64
dtype: object

In [64]:
application_df.nunique()

APPLICATION_TYPE    9
AFFILIATION         6
CLASSIFICATION      6
USE_CASE            5
ORGANIZATION        4
INCOME_AMT          9
ASK_AMT             5
IS_SUCCESSFUL       2
dtype: int64

In [65]:
preprocessed_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 
                                        'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT'])
preprocessed_df

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,"ASK_AMT_(0.0, 5000.0]","ASK_AMT_(5000.0, 100000.0]","ASK_AMT_(100000.0, 1000000.0]","ASK_AMT_(1000000.0, 5000000.0]","ASK_AMT_(5000000.0, inf]"
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34295,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34296,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34297,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [66]:
# Split our preprocessed data into our features and target arrays
target = preprocessed_df['IS_SUCCESSFUL']
features = preprocessed_df.drop('IS_SUCCESSFUL', axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

In [67]:
features.shape


(34299, 44)

In [68]:
target

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [69]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## M1: Compile, Train and Evaluate the Model

In [73]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
dense = tf.keras.layers.Dense(input_dim=44, units=80, activation="relu")
nn.add(dense)

# Second hidden layer
dense1 = tf.keras.layers.Dense(units=30, activation="relu")
nn.add(dense1)

# Output layer
dense2 = tf.keras.layers.Dense(units=1, activation="sigmoid")
nn.add(dense2)

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 80)                3600      
                                                                 
 dense_34 (Dense)            (None, 30)                2430      
                                                                 
 dense_35 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,061
Trainable params: 6,061
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [76]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7377
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5380 - accuracy: 0.7369
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5379 - accuracy: 0.7388
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5376 - accuracy: 0.7382
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5373 - accuracy: 0.7385
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5367 - accuracy: 0.7393
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5366 - accuracy: 0.7381
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7394
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5360 - accuracy: 0.7399
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5355 - accura

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5648 - accuracy: 0.7272 - 444ms/epoch - 2ms/step
Loss: 0.564752459526062, Accuracy: 0.7272303104400635


In [78]:
nn.save('AlphabetSoupCharity_opt1.h5')

#Model 2: Add more hidden layers and neurons

In [79]:
#ATTEMPT 2 - add more neurons and layers
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

n2 = tf.keras.models.Sequential()

# First hidden layer
dense = tf.keras.layers.Dense(input_dim=44, units=200, activation="relu")
n2.add(dense)

# Second hidden layer
dense1 = tf.keras.layers.Dense(units=100, activation="relu")
n2.add(dense1)

# third hidden layer
dense2 = tf.keras.layers.Dense(units=64, activation="relu")
n2.add(dense2)

# fourth hidden layer
dense3 = tf.keras.layers.Dense(units=32, activation="relu")
n2.add(dense3)

# fifth hidden layer
dense4 = tf.keras.layers.Dense(units=16, activation="relu")
n2.add(dense4)


# Output layer
denseoutput = tf.keras.layers.Dense(units=1, activation="sigmoid")
n2.add(denseoutput)

# Check the structure of the model
n2.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 200)               9000      
                                                                 
 dense_37 (Dense)            (None, 100)               20100     
                                                                 
 dense_38 (Dense)            (None, 64)                6464      
                                                                 
 dense_39 (Dense)            (None, 32)                2080      
                                                                 
 dense_40 (Dense)            (None, 16)                528       
                                                                 
 dense_41 (Dense)            (None, 1)                 17        
                                                                 
Total params: 38,189
Trainable params: 38,189
Non-trai

In [80]:
n2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [81]:
fit_model_2 = n2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5689 - accuracy: 0.7218
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5542 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5503 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5486 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5471 - accuracy: 0.7328
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accuracy: 0.7346
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5449 - accuracy: 0.7344
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5433 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5428 - accuracy: 0.7357
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accura

In [85]:
# Evaluate the model using the test data
model_loss, model_accuracy = n2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6124 - accuracy: 0.7294 - 507ms/epoch - 2ms/step
Loss: 0.6123952269554138, Accuracy: 0.7294460535049438


In [86]:
n2.save('AlphabetSoupCharity_opt2.h5')

##Model 3: Add More Epochs and Try Stochastic Gradient Descent Optimizer


In [87]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n3 = tf.keras.models.Sequential()

# First hidden layer
dense = tf.keras.layers.Dense(input_dim=44, units=200, activation="relu")
n3.add(dense)

# Second hidden layer
dense1 = tf.keras.layers.Dense(units=100, activation="relu")
n3.add(dense1)

# third hidden layer
dense2 = tf.keras.layers.Dense(units=50, activation="relu")
n3.add(dense2)

# fourth hidden layer
dense3 = tf.keras.layers.Dense(units=20, activation="relu")
n3.add(dense3)

# Output layer
denseoutput = tf.keras.layers.Dense(units=1, activation="sigmoid")
n3.add(denseoutput)

# Check the structure of the model
n3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 200)               9000      
                                                                 
 dense_48 (Dense)            (None, 100)               20100     
                                                                 
 dense_49 (Dense)            (None, 50)                5050      
                                                                 
 dense_50 (Dense)            (None, 20)                1020      
                                                                 
 dense_51 (Dense)            (None, 1)                 21        
                                                                 
Total params: 35,191
Trainable params: 35,191
Non-trainable params: 0
_________________________________________________________________


In [88]:
n3.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
fit_model_3 = n3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6242 - accuracy: 0.6889
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5622 - accuracy: 0.7273
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7280
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5518 - accuracy: 0.7293
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7309
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7318
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7310
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7336
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accuracy: 0.7332
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accura

In [89]:
# Evaluate the model using the test data
model_loss, model_accuracy = n3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5646 - accuracy: 0.7252 - 1s/epoch - 5ms/step
Loss: 0.5646436810493469, Accuracy: 0.7252478003501892


In [90]:
# Export our model to HDF5 file
n3.save('AlphabetSoupCharity_opt3.h5')